In [1]:
import urllib
import warnings
from pathlib import Path as p
from pprint import pprint

import pandas as pd
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import VertexAIEmbeddings
from langchain.llms import VertexAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from IPython.display import display, Markdown

warnings.filterwarnings("ignore")

In [2]:
# from langchain.document_loaders import PyPDFLoader
# from langchain.document_loaders import CSVLoader
# file = 'OutdoorClothingCatalog_1000.csv'
# loader1 = CSVLoader(file_path=file)
# pdfFile = '/OfferCreationviaGrumbles.pdf'

# loader1

In [2]:
import pathlib
desktop = pathlib.Path("./SOP")
files = []
list(desktop.iterdir())
pages1 = []
# for item in list(desktop.iterdir())[:1]:
for item in list(desktop.iterdir()):
    try:
        print(str(item))
        fileLoader = PyPDFLoader(str(item))
#         pages = fileLoader.load_and_split()
#         print(len(fileLoader.load_and_split()))
        pages1.append(fileLoader.load_and_split())
#         files.append(pages)
    except:
        print("An exception occurred")
  
print (type(pages1))


SOP/Incentivised Attach SOP.pdf
SOP/Targeted Discounts - SOP.pdf
SOP/Bump Up Offers.pdf
SOP/Casper SOP.pdf
SOP/Targeted Discount on FP - SOP.pdf
SOP/_SOP for Shipping Rule Creation.pdf
SOP/Exchange Offers.pdf
SOP/Prebook Offer.pdf
SOP/Shipping Offers - SOP.pdf
SOP/Either Or offers SOP.pdf
SOP/Promise Callout Offers - SOP.pdf
SOP/BMGN v1 Offer SOP.pdf
SOP/Callout Offer.pdf
SOP/CnC Offers - SOP.pdf
SOP/Multi Unit Listing Offers - SOP.pdf
SOP/Seller Recommendation for an Offer.pdf
SOP/Offer Creation via Grumbles .pdf
SOP/Basket Offers - SOP (1).pdf
SOP/Early Access SOP.pdf
SOP/Combo Offers - SOP.pdf
SOP/PBO SOP.pdf
<class 'list'>


In [26]:


pages = []
print(len(pages1))
for item in pages1:
    try:
        if(item.page_content):
            pages.append(item)
            print(item)
    except:
        for item2 in item:
            pages.append(item2)
print(len(pages))

context = "\n".join(str(p.page_content) for p in pages[:7])
print(type(context))
print(len(context))
print(context)

21
207
<class 'str'>
9745
W h a t
a r e
I n c e n t i v i s e d
A t t a c h
O ﬀ e r s ?
B u y
P 1 ,
g e t
x %
o ﬀ
o n
P 2
I n c e n t i v i s e d
A t t a c h
O ﬀ e r s
a r e
m u l t i
p r o d u c t
g r o u p
o ﬀ e r s
t h a t
h a v e
a
h u r d l e
o n
1
p r o d u c t
g r o u p ,
a n d
a
g i v e a w a y
o n
a
d i ﬀ e r e n t
p r o d u c t
g r o u p .
P r o d u c t
g r o u p
r e f e r s
t o
a
s e t
o f
F S N s
d e ﬁ n e d
b y
t h e
o ﬀ e r
c r e a t o r .
I f
t h e
c o m b o
o ﬀ e r
h a s
2
P G s ,
a
u s e r
n e e d s
t o
b u y
a n
i t e m
f r o m
e a c h
o f
t h e
P G
w h i c h
w i l l
m a k e
h i m / h e r
e l i g i b l e
f o r
d i s c o u n t
o n
t h e
s e c o n d
P G .
E . g .
B u y
m o b i l e s ,
g e t
1 0 %
o ﬀ
o n
e a r p h o n e s
P G 1
-
M o b i l e s
P G
2
-
E a r p h o n e s
I f
a
c u s t o m e r
p u r c h a s e s
b o t h
1
m o b i l e
a n d
1
e a r p h o n e s
i n
t h e
s a m e
t r a n s a c t i o n ,
t h e y
w i l l
g e t
1 0 %
o ﬀ
o n
t h e
e a r p h o n e s
l i s t i n g 

In [4]:
vertex_llm_text = VertexAI(model_name="text-bison@001")
vertex_embeddings = VertexAIEmbeddings(model_name="textembedding-gecko@001")

In [22]:
# question = "What is Incentivised Attach?"
question = "What is Insufficient Participation?"
prompt_template = """Answer the question as precise as possible using the provided context. If the answer is
                    not contained in the context, say "answer not available in context" \n\n
                    Context: \n {context}?\n
                    Question: \n {question} \n
                    Answer:
                  """

prompt = PromptTemplate(
    template=prompt_template, 
    input_variables=["context", "question"]
)

In [23]:
stuff_chain = load_qa_chain(vertex_llm_text, chain_type="stuff", prompt=prompt)

In [24]:
stuff_answer = stuff_chain(
    {"input_documents": pages[:7], "question": question}, 
    return_only_outputs=True
)

In [25]:
print(stuff_answer)

{'output_text': ' The Insufficient Participation is a situation where the number of users who have mapped to the offer is less than the minimum required mapping count.'}


In [13]:
# !pip install transformers

In [14]:
final_mp_data = []

# for each document, extract metadata and intermediate steps of the MapReduce process
for doc, out in zip(
    map_reduce_outputs["input_documents"], map_reduce_outputs["intermediate_steps"]
):
    output = {}
    output["file_name"] = p(doc.metadata["source"]).stem
    output["file_type"] = p(doc.metadata["source"]).suffix
    output["page_number"] = doc.metadata["page"]
    output["chunks"] = doc.page_content
    output["answer"] = out
    final_mp_data.append(output)

In [16]:
# create a dataframe from a dictionary
pdf_mp_answers = pd.DataFrame.from_dict(final_mp_data)
# sorting the dataframe by filename and page_number
pdf_mp_answers = pdf_mp_answers.sort_values(by=["file_name", "page_number"])
pdf_mp_answers.reset_index(inplace=True, drop=True)
pdf_mp_answers.head()
print(pdf_mp_answers)

                 file_name file_type  page_number  \
0  Incentivised Attach SOP      .pdf            0   
1  Incentivised Attach SOP      .pdf            1   

                                              chunks  \
0  W h a t\na r e\nI n c e n t i v i s e d\nA t t...   
1  O ﬀ e r\nD e t a i l s\nS t e p\n1 :\nD o\ny o...   

                                              answer  
0  If the number of items sold in the first produ...  
1  If the seller burn share is less than 100%, th...  


In [ ]:
# !pip install docarray
from langchain.vectorstores import DocArrayInMemorySearch
db = DocArrayInMemorySearch.from_documents(
    pages, 
    vertex_embeddings
)

In [ ]:
# print(type(pages))

In [ ]:
### we will use above vectorstore to find answer to query
# query = "What is Incentivised offer"

In [ ]:
## find all similar document
# docs = db.similarity_search(query)
# print(len(docs))
## 4 docs are returned
# docs[0] ## indeed a shirt

In [ ]:
retriever = db.as_retriever()

In [ ]:
from vertexai.preview.language_models import TextGenerationModel


def getResponse(prompt ,temperature: float = 0.2):
    """Ideation example with a Large Language Model"""

    # TODO developer - override these parameters as needed:
    parameters = {
        "temperature": temperature,  # Temperature controls the degree of randomness in token selection.
        "max_output_tokens": 256,  # Token limit determines the maximum amount of text output.
        "top_p": 0.8,  # Tokens are selected from most probable to least until the sum of their probabilities equals the top_p value.
        "top_k": 40,  # A top_k of 1 means the selected token is the most probable among all tokens.
    }

    model = TextGenerationModel.from_pretrained("text-bison@001")
    response = model.predict(
        prompt,
        **parameters,
    )
    
    return response


In [ ]:

# from vertexai.preview.language_models import ChatModel, InputOutputTextPair

# def getChatResponse(prompt, temperature: float = 0.2):
#     chat_model = ChatModel.from_pretrained("chat-bison@001")

#     # TODO developer - override these parameters as needed:
#     parameters = {
#         "temperature": temperature,  # Temperature controls the degree of randomness in token selection.
#         "max_output_tokens": 256,  # Token limit determines the maximum amount of text output.
#         "top_p": 0.95,  # Tokens are selected from most probable to least until the sum of their probabilities equals the top_p value.
# #         "top_k": 40,  # A top_k of 1 means the selected token is the most probable among all tokens.
#     }

#     chat = chat_model.start_chat(
#         context=SOPContext,
#         examples=[
#             InputOutputTextPair(
#             ),
#         ],
#     )

#     response = chat.send_message(
#         query, **parameters
#           )
#     print(f"Response from Model: {response.text}")


In [ ]:
# query = "how does deferred Giveaway works for offer"
# query = "how does I create a Promise Callout Offer"
query = "What is insufficient participation ?"
relevantDocs = db.max_marginal_relevance_search(query, k=20)
# relevantDocs = db.similarity_search_with_score(query, k=10)

print(len(relevantDocs))
# print(relevantDocs)

qdocs = "".join([relevantDocs[i].page_content for i in range(len(relevantDocs))])
# print(qdocs)
for i in range(0,len(qdocs)-2):
    print(relevantDocs[i])
    relevantDocs[i].page_content
    print(item, "======\n")

In [ ]:

role = """
You are offer related query answering module for promotions platform on Flipkart.\
You answer questions with a high degree of precision & wherever you do not have enough precision, \
you always provide an output saying the answers might not be fully correct. \
Promotions Platform has capabilities to create and maintain an offer via its \
offer creation console- 'Grumbles' and maintenance console-- 'Casper'. Business users\
can ask multiple questions related to capabilities for creating and maintaining an offer.
"""

prompt = f""" 
Role: {role}
Follow exactly these 3 steps:
1. Read the context below and aggregrate this data
Context : {qdocs}
2. Answer the question using only this context
3. Show the source for your answers
User Question: {query}

If you don't have any context and are unsure of the answer, reply that you don't know about this topic.

"""
print(len(prompt))

In [ ]:
print(getResponse(prompt))

In [ ]:

# response = vertex_llm_text.call_as_llm(f"{qdocs} Question: What is Incentivised offer.") 

# print (sopContext)

# print(prompt)
# question =  f"{qdocs} {prompt}"
# print(question)
# print(getResponse(question))
# display(Markdown(response))

# getChatResponse()

In [ ]:
!gcloud auth application-default login